In [1]:
import sys
sys.path.append('../')
from src.Algorithms.ReLUToTLogic import *
from src.Algorithms.TLogicToReLU import *
from src.Logic.Solver import *

sys.setrecursionlimit(10**6)

# Simple examples jupyter notebook

This notebook illustrate how to properly use the pipeline

## XOR Network

In [2]:
XOR_ReLU = ReLUNetwork(
    weights=[
        torch.tensor([[1, -1], [-1, 1]], dtype=torch.float64),
        torch.tensor([[1, 1]], dtype=torch.float64)
    ],
    biases=[
        torch.tensor([0, 0], dtype=torch.float64),
        torch.tensor([0], dtype=torch.float64)
    ]
)

XOR_ReLU.construct_layers()

# Let's check the output of the network against the truth table
for x1 in [0, 1]:
    for x2 in [0, 1]:
        x = torch.tensor([x1, x2], dtype=torch.float64)
        print(f"XOR({x1}, {x2}) = {XOR_ReLU.forward(x)}")

XOR(0, 0) = tensor([0.], dtype=torch.float64, grad_fn=<ViewBackward0>)
XOR(0, 1) = tensor([1.], dtype=torch.float64, grad_fn=<ViewBackward0>)
XOR(1, 0) = tensor([1.], dtype=torch.float64, grad_fn=<ViewBackward0>)
XOR(1, 1) = tensor([0.], dtype=torch.float64, grad_fn=<ViewBackward0>)


Let's check the formula for this network. It is well know that the formula $((x_1∧(¬x_2))\lor((¬x_1)∧x_2))$.

In [3]:
CReLU = transform_ReLU_to_CReLU(XOR_ReLU)

transformed_formula = CReLU_to_formula(CReLU)[0]
original_formula = "((x1∧(¬x2))V((¬x1)∧x2))"

print(transformed_formula)

# Let's check the output of the network against the truth table
prsr = Parser(Lukasiewicz())
for x1 in [0, 1]:
    for x2 in [0, 1]:
        transformed_formula_ast = prsr.generate_ast_with_degs(transformed_formula)[0]
        original_formula_ast = prsr.generate_ast_with_degs(original_formula)[0]


        print(f"XOR({x1}, {x2}) = {Parser.evaluate_formula(prsr, transformed_formula_ast, {'x1': x1, 'x2': x2})}")
        print(f"XOR({x1}, {x2}) = {Parser.evaluate_formula(prsr, original_formula_ast, {'x1': x1, 'x2': x2})}")

((x2⊙(¬x1))⊕(x1⊙(¬x2)))
XOR(0, 0) = 0.0
XOR(0, 0) = 0.0
XOR(0, 1) = 1.0
XOR(0, 1) = 1.0
XOR(1, 0) = 1.0
XOR(1, 0) = 1.0
XOR(1, 1) = 0.0
XOR(1, 1) = 0.0


As we can see, the formulas are equivalent in Lukasiewicz logic. Let's now check for reachability

In [4]:
SolveFormulaSMT(transformed_formula, 1)

(True, [x2 = 0, x1 = 1])

As we can see the formula is solvable (something we already knew), and we are given a correct model

One of the things that would be fun to check are the formulas of the neurons and check if we can learn anything from them

In [5]:
_, neuron_formulas = CReLU_to_formula(CReLU)

first_layer_formulas = neuron_formulas[0]
second_layer_formulas = neuron_formulas[1]

print("First layer formulas:")
print(first_layer_formulas[1])
print(first_layer_formulas[2] + "\n")
print("Second layer formulas:")
print(second_layer_formulas[1])

First layer formulas:
(x1⊙(¬x2))
(x2⊙(¬x1))

Second layer formulas:
((x2⊙(¬x1))⊕(x1⊙(¬x2)))


As we can see, the first formula is only true when $x_1$ is true and $x_2$ is false, which is the case $(1,0)$. As for the second formula, it is only true when $x_1$ is false and $x_2$ is true, which is the case $(0,1)$.

The final layer computes the "OR" part of XOR, being true on input $(1,0)$ or $(0,1)$

Therefore, using this algorithm, we can explain exactly what each neuron outputs in this simple example